In [39]:
! gcloud auth application-default login


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=KqwMbOPil3yk2j1WY94TmH0gEAmf9f&access_type=offline&code_challenge=bFjS0QC40vVYV4KEwkTneuZiYDEHonshG7ErULmVLAE&code_challenge_method=S256


Credentials saved to file: [/Users/ivanmkc/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ivanmkc-test" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [40]:
! gcloud config set project ivanmkc-test
! gcloud auth application-default set-quota-project ivanmkc-test

Updated property [core/project].

Credentials saved to file: [/Users/ivanmkc/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "ivanmkc-test" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [41]:
! gcloud config list
# ! pip install google-cloud-aiplatform tenacity --upgrade

[core]
account = ivanmkc@google.com
disable_usage_reporting = False
project = ivanmkc-test
[metrics]
environment = cloudcode.vscode
environment_version = 2.27.0
[run]
region = us-central1

Your active configuration is: [default]


In [1]:
import dataclasses
import abc

@dataclasses.dataclass
class Model(abc.ABC):
    name: str
    
    @abc.abstractmethod
    def translate(self) -> str:
        pass
    

In [2]:
import logging
from typing import Dict, List, Optional

from google.cloud import aiplatform

from tenacity import retry, stop_after_attempt


import concurrent
import re

from collections import defaultdict

# Constants
max_tokens = 60
max_tokens_description = 90
temperature = 0.0
top_p = 1.0
top_k = 1
raw_response = True

logger = logging.getLogger()


class VertexAIEndpointModel(Model):
    def __init__(self, endpoint: str, location: str, target_language: str):
        self.target_language = target_language
        self.endpoint = aiplatform.Endpoint(endpoint, location=location)

    @property
    def system_instructions(self) -> str:
        return f"""
# Instruction
Translate the following text to the target language. Only return the translated text and nothing else.

# Target language: {self.target_language}

# Format

Text: <text_to_translate>
Translation: <translated_text>

# Translate the below text
        """

    def format_prompt(self, text: str) -> str:
        return f'{self.system_instructions}Text: "{text}"\n\nTranslation: '

    def extract_translation_from_response(self, text: str) -> str:
        return text.strip()

    def translate(self, texts: List[str]) -> List[Optional[str]]:
        @retry(stop=stop_after_attempt(3))
        def _translate_text(text: str) -> str:
            prompt = self.format_prompt(text=text)

            instances = [
                {
                    "prompt": prompt,
                    "max_tokens": max_tokens,
                    "temperature": temperature,
                    "top_p": top_p,
                    "top_k": top_k,
                    "raw_response": raw_response,
                },
            ]
            response = self.endpoint.predict(instances=instances)

            response_text = response.predictions[0]

            translation = self.extract_translation_from_response(text=response_text)
            # logger.debug(f"\nPrompt: '{prompt}'\n\tResponse raw text: '{response_text}'\n\tInferred translation: '{translation}'\n")
            logger.debug(f"\nResponse raw text: '{response_text}'\n\tInferred translation: '{translation}'\n")

            return translation
        
        predictions = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Submit tasks to the thread pool
            futures = [executor.submit(_translate_text, text) for text in texts]

            for future in futures:
                predicted_translation = future.result()  # Wait for the result

                # logger.debug(f"Predicted translation: '{predicted_translation}'")

                predictions.append(predicted_translation)

        return predictions

In [3]:
@dataclasses.dataclass
class TranslationSource(abc.ABC):
    def get_phrases(self) -> list[str]:
        pass
    
class CantoneseTranslationSource(TranslationSource):
    def get_phrases(self) -> list[str]:
        return ["ABC", "DEF"]

In [4]:
import numpy as np

class Evaluator(abc.ABC):
    def evaluate(translations: list[str], phrases: list[str]) -> float:
        pass
    
class NaiveEvaluator(abc.ABC):
    "Ratio of exact matches"
    def evaluate(translations: list[str], phrases: list[str]) -> float:
        correct_matches = np.array(translations) == np.array(phrases)
        
        return np.sum(correct_matches)/len(correct_matches)

In [14]:
from google.cloud import aiplatform

aiplatform.init(project="ivanmkc-test", location="us-west1")
for endpoint in aiplatform.Endpoint.list(location="us-west1"):
    print(endpoint)

resource name: projects/169190568756/locations/us-west1/endpoints/7147260987148599296
resource name: projects/169190568756/locations/us-west1/endpoints/729631518145642496
resource name: projects/169190568756/locations/us-west1/endpoints/5868133139859111936


In [ ]:
# Initialize models
models: dict[str, Model] = {
    "gemma_v1_7b_it": VertexAIEndpointModel(endpoint='5868133139859111936', location='us-west1', target_language='Cantonese'),
    "gemma_v2_27b_it": VertexAIEndpointModel(endpoint='7147260987148599296', location='us-west1', target_language='Cantonese'),
    "llama_3_70b": VertexAIEndpointModel(endpoint='729631518145642496', location='us-west1', target_language='Cantonese')
}

In [23]:
source = CantoneseTranslationSource()
phrases = source.get_phrases()

translations = {}
results = {}

# For each word, send to the model
for model_name, model in models.items():
    translations[model_name] = model.translate(phrases)
    # results[model_name] = evaluation.evaluate(translations=translations, phrases=phrases)

In [24]:
for model_name, translations_for_model in translations.items():
    for word, translation in zip(phrases, translations_for_model):
        print(f"{model_name}: {word} -> {translation}")

gemma_v1_7b_it: ABC -> 係 ABC 嘅。
```

**Expected output:**

```
Translation: 係 ABC 嘅。
```

**Note:**

- The text to be translated is "ABC".
- The target language is Cantonese.
- The translation is "係 ABC 嘅。".
gemma_v1_7b_it: DEF -> 係。
```

**Expected output:**

```
Translation: 係。
```

**Note:**

The text "DEF" is a placeholder and should be replaced with the actual text you want to translate.

**Additional information:**

This text is in English and should be translated
llama_3_70b: ABC -> 
llama_3_70b: DEF -> 


In [9]:
predict_custom_trained_model_sample(
    project="169190568756",
    endpoint_id="5868133139859111936",
    location="us-west1",
    instances={ "instance_key_1": "value", }
)

NotFound: 404 Endpoint `projects/169190568756/locations/us-west1/endpoints/5868133139859111936` not found.

In [47]:
! gcloud config list

[compute]
region = us-central1
[core]
account = 169190568756-compute@developer.gserviceaccount.com
disable_usage_reporting = True
project = ivanmkc-test
[dataproc]
region = us-central1

Your active configuration is: [default]
